In [4]:
import csv
import os
import json

import numpy as np

In [5]:
def load_csv(csv_path):
    loaded_data = []
    with open(csv_path, "r") as csv_file:
        csv_reader = csv.reader(csv_file,delimiter=',')
        for i, row in enumerate(csv_reader):
            if i == 0:
                header_info = row
            else:
                current_data_point = {}
                for j, x in enumerate(row):
                    current_data_point[header_info[j]] = x
                loaded_data.append(current_data_point)
    return loaded_data

In [6]:
def save_csv(visualization, output_path):
    #csv_columns = ['original_id', "original_obj", "original_caption", "retrieved_1", "retrieved_2", "retrieved_3", "original_url", "retrieved_url_1"]
    csv_columns = list(visualization[0].keys())
    with open(output_path, "w") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in visualization:
            writer.writerow(data)

In [7]:
with open("/data/home/zmykevin/vinvl_data/CC/cc_objects_captions_sorted.json", "r") as f:
    cc_objects_captions_sorted = json.load(f)

with open("/data/home/zmykevin/vinvl_data/CC/cc_objects_captions.json", "r") as f:
    cc_objects_captions = json.load(f)

In [8]:
valid_count = 0
for cc_a, cc_b in zip(cc_objects_captions_sorted.values(), cc_objects_captions.values()):
    cc_a_objects = cc_a['objects_no_rep']
    cc_b_objects = cc_b['objects_no_rep']
    print(cc_a_objects)
    print(cc_b_objects)
    print("\n")
    valid_count +=1 
    if valid_count == 5:
        break
    

road bus street building car man person
pant street building shirt man person hat car roof sign door van window road balcony boat bus


poster woman drum girl person dress wall leg shirt skirt man hair arm
man neck wall woman shirt skirt face top shoe arm person poster hand girl dress tripod drum hair leg picture microphone


closet floor clothes cabinet piano scarf woman tub bathtub shirt window fence jean hanger
bathtub scarf pillow glasses hanger handle jean woman shirt bag clothing window floor hand girl closet clothes piano tub door rug drawer fence hair cabinet


building wall room brick ceiling floor fireplace desk bar shadow ledge log fire
floor ceiling fireplace building room bar log desk brick word shadow ledge fire sign tile letter bench wall light


sky leg boot string word handlebar
string word foot handle letter handlebar sky leg boot




In [10]:
#Load the Original Annotation:
original_cc_val_path = "/data/home/zmykevin/vinvl_data/CC/Validation_GCC-1.1.0-Validation.tsv"
original_cc_train_path = "/data/home/zmykevin/vinvl_data/CC/Train_GCC-training.tsv"
val = []
train = []
with open(original_cc_val_path) as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        val.append(row)

with open(original_cc_train_path) as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        train.append(row)
print(val[0])
print(train[0])

['author : a life in photography -- in pictures', 'https://i.pinimg.com/736x/66/01/6c/66016c3ba27c0e04f39e2bd81a934e3e--anita-ekberg-bob-hope.jpg']
['a very typical bus station', 'http://lh6.ggpht.com/-IvRtNLNcG8o/TpFyrudaT6I/AAAAAAAAM6o/_11MuAAKalQ/IMG_3422.JPG?imgmax=800']


In [11]:
#Get the caption -> url mapping
caption_url_mapping = {}
for x in val+train:
    caption = x[0]
    url = x[1]
    caption_url_mapping[caption] = url

In [12]:
data_path = "/data/home/zmykevin/vinvl_data/CC"
csv_path = os.path.join(data_path, "captions_retrieved_bertembedding_bookcorpus_sorted.csv")

loaded_data = load_csv(csv_path)

In [14]:
loaded_data_sub = loaded_data[:1000]
loaded_data_sub_updated = []
for x in loaded_data_sub:
    #print(x)
    #original_caption = x['original_caption']
    retrieved_1_caption = x['retrieved_1']
    #original_url = caption_url_mapping[original_caption]
    #retrieved_url = caption_url_mapping[retrieved_1_caption]
    #x['original_url'] = original_url
    #x['retrieved_url_1'] = retrieved_url
    loaded_data_sub_updated.append(x)
print(len(loaded_data_sub_updated))

1000


In [15]:
print(loaded_data_sub_updated[0])
save_csv(loaded_data_sub_updated, os.path.join(data_path, "captions_retrieved_bertembedding_sorted_bookcorpus_1000.csv"))

{'original_id': '0', 'original_obj': '', 'original_caption': 'the half-ling book one in the fall of igneeria series kaylee soderburg copyright 2013 kaylee soderburg all rights reserved .', 'retrieved_1': 'on the other side of the road a worried looking passenger boarded a bus and nodded .', 'retrieved_2': 'the bus driver said .', 'retrieved_3': 'as he circled around the bus this time , he caught a glimpse of a black-clothed figure moving from the office to the warehouse .'}


In [41]:
#Visualize the Best and Worst CLIP Scores using each method;
clip_score_path = "/data/home/zmykevin/vinvl_data/CC/cc_clip_score_tfidf.json"
with open(clip_score_path, "r") as f:
    clip_score = json.load(f)

In [5]:
cc_caption_objects_path = "/data/home/zmykevin/vinvl_data/CC/cc_objects_captions.json"
with open(cc_caption_objects_path, "r") as f:
    cc_caption_objects = json.load(f)

In [42]:
clip_score_ids = []
clip_score_list = []
for k,v in clip_score.items():
    clip_score_ids.append(k)
    clip_score_list.append(v)

In [43]:
clip_score_arg = np.argsort(np.array(clip_score_list))

In [44]:
best_scores_id = [clip_score_ids[x] for x in clip_score_arg[-10:]]
worst_scores_id = [clip_score_ids[x] for x in clip_score_arg[:10]]
print(best_scores_id)
print(worst_scores_id)

['16503', '23133', '12732', '4012', '31495', '31830', '31300', '31085', '25412', '28371']
['34063', '28646', '27320', '1646', '15691', '7035', '35159', '7513', '26745', '30458']


In [45]:
#Visualize these examples
retrieved_caption_path = "/data/home/zmykevin/vinvl_data/CC/captions_retrieved_bertembedding_top10_sorted.json"
with open(retrieved_caption_path, "r") as f:
    retrieved_captions = json.load(f)

In [48]:
score_log = []
for original_id in best_scores_id:
    original_objects = cc_caption_objects[original_id]["objects_no_rep"]
    original_caption = cc_caption_objects[original_id]["caption"]
    retrieved_id = retrieved_captions[original_id][0]
    retrieved_caption = cc_caption_objects[retrieved_id]["caption"]
    
    original_image_url = caption_url_mapping[original_caption]
    retrieved_image_url = caption_url_mapping[retrieved_caption]
    score_log.append({"original_id": original_id, 
                           "original_objects": original_objects, 
                           "original_caption": original_caption, 
                           "retrieved_caption": retrieved_caption, 
                           "original_image_url": original_image_url, 
                           "retrieved_image_url": retrieved_image_url})


In [49]:
save_csv(score_log, "/data/home/zmykevin/vinvl_data/CC/best_captions_tfidf.csv")